In [1]:
import os
from datetime import datetime
import pickle
import numpy as np
import tqdm
import json
from datetime import datetime
progress_bar = True
import time
import dynetx as dn
import openMindednessFunctions as omf

# Estimating open mindedness distributions + other info

In [2]:
def createDataset(t0, tN):
    data = {}
    for dataset_name in ['euro2020']:
        '''
        per ogni time step mi ritorna un dizionario nodo: {dizionario con delle info}
        '''
        data[dataset_name] = {}
        timestamps = {0: 't0', 1: 't1'}
        t2node2opinions = omf.create_dictionaries(timestamps, '../dataset')
        for t in tqdm.tqdm(range(t0, tN)):
            data[dataset_name][t] = {}
            t1=t+1
            g = omf.createGraph('../dataset', timestamps, t)
            for v in tqdm.tqdm(g.nodes()):
                if v in t2node2opinions[t1].keys():
                    opvt = t2node2opinions[t][v]
                    opvt1 = t2node2opinions[t1][v]
                    neighs = list(g.neighbors(v))
                    sortedNeighOps = omf.sortNeighOps(g, v, t, t2node2opinions, opvt)
                    eps, err, estOp = omf.estimation(opvt, opvt1, sortedNeighOps)
                    orientation = omf.politicalLeaning(opvt) 
                    nactiveint = len([op for op in sortedNeighOps if op <= eps])
                    data[dataset_name][t][v] = {'dataset':dataset_name, 'opt':opvt, 'opt1': opvt1, 'neighbors':neighs, 
                                                    'opChange':opvt1-opvt, 'orientation':orientation, 
                                                    'neighborsOps':sortedNeighOps, 'error':err, 'estOp':estOp, 
                                                    'openMindedness':eps, 'nactiveint':nactiveint}    
                    if t > t0:
                        try:
                            data[dataset_name][t][v]['orientation-1'] = data[t-1][v]['orientation']
                        except:
                            data[dataset_name][t][v]['orientation-1'] = None
    return data

In [3]:
data = createDataset(0, 1)

100%|██████████| 1/1 [00:00<00:00, 10.15it/s]


In [4]:
with open(f'openMindednessData.pickle', 'wb') as ofile:
    pickle.dump(data, ofile)